In [3]:
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

import pickle

from IPython import display
%matplotlib inline

from tqdm import tqdm_notebook

In [4]:
from keras.layers import Dense, Embedding

In [166]:
class GraphConvLayer:
    
    def __init__(self, name, window, size):
        self.name = name
        self.window = window
        self.size = size
        
        with tf.variable_scope(name) as scope:
            self.g = tf.get_variable('g',
                                           shape = [size, window],
                                           initializer=None, trainable=True)
            
            self.b = tf.get_variable('b' , shape = [size], initializer=None, trainable=True) 
        self.scope = scope
        
    def __call__(self, inp, a_powers):
        assert a_powers.shape[0] == self.window
        
        with tf.variable_scope(self.scope):
            with tf.name_scope(self.scope.original_name_scope):
                tm = tf.tensordot(a_powers, inp, axes = [[2], [0]])
                tm_sum = tf.reduce_sum(tm, 2)

                out = tf.tensordot(tm_sum, self.g, axes=[[1],[1]]) + self.b
                
        return out                

In [143]:
A = np.array([[0., 1], [2, 3]]).astype(np.float32)

A_power = np.stack((A, np.dot(A, A)))

In [144]:
x_test = np.array([[1., 2, 3, 4],[1, 2, 3, 4]])

In [145]:
A_power

array([[[ 0.,  1.],
        [ 2.,  3.]],

       [[ 2.,  3.],
        [ 6., 11.]]], dtype=float32)

In [146]:
x_test

array([[1., 2., 3., 4.],
       [1., 2., 3., 4.]])

In [167]:
tf.reset_default_graph()
sess = tf.Session()

In [168]:
inp = tf.placeholder(tf.float32, [None, None])

cl = GraphConvLayer('322', 2, 3)

out = cl(inp, A_power)

In [156]:
g = np.array([[0.1, 0.5], [1, 2], [4, 8]]).astype(np.float32)

b = np.array([-100, -1000, -10000])

In [157]:
g.shape

(3, 2)

In [158]:
tm = tf.tensordot(A_power, inp, axes = [[2], [0]])
tm_sum = tf.reduce_sum(tm, 2)

tm_2 = tf.tensordot(tm_sum, g, axes=[[1],[1]]) + b

In [171]:
sess.run(out, feed_dict={inp : x_test})


array([[ -1.9776053,  24.320324 ,  38.344852 ],
       [-17.89899  ,  77.487656 , 120.48454  ]], dtype=float32)

In [170]:
sess.run(tf.global_variables_initializer())

In [163]:
sess.run([cl.g, cl.b])

[array([[-0.11203653,  0.6464416 ],
        [ 0.31079972,  1.064425  ],
        [ 0.41860187,  0.4798901 ]], dtype=float32),
 array([-0.13279462,  0.3436036 ,  0.50527596], dtype=float32)]

In [63]:
A_power

array([[[ 0.,  1.],
        [ 2.,  3.]],

       [[ 2.,  3.],
        [ 6., 11.]]], dtype=float32)

In [61]:
sess.run(out, feed_dict={inp:x_test})

array([[-1.5545536],
       [-4.277301 ]], dtype=float32)

In [56]:
gr = tf.get_default_graph()

writer = tf.summary.FileWriter("../tf_logs", gr)

In [165]:
(-0.11 * np.dot(A_power[0], x_test) + 0.646 * np.dot(A_power[1], x_test)).sum(1)

array([ 31.2 , 104.32])